# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol='AAPL'
api_endpoint = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_endpoint).json()

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']
print(price)
print(market_cap)

182.9
2978159773680


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
df1 = pd.DataFrame(columns=my_columns)
df1

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [29]:
# Notice importance of ignore_index attribute in the next cell
pd.concat([df1,pd.Series([
    symbol,
    price,
    market_cap,
    'N/A'
], index=my_columns).to_frame().T])

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,182.9,2978159773680,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [31]:
df = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker'][:5]:
    api_endpoint = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_endpoint).json()
    df = pd.concat([df,pd.Series([
        stock,
        data['latestPrice'],
        data['marketCap'],
        "N/A"
        ], index=my_columns).to_frame().T], ignore_index=True)

df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,138.35,41115767471,N/A
1,AAL,17.95,11692087082,N/A
2,AAP,218.42,12946765300,N/A
3,AAPL,180.8,2919847464372,N/A
4,ABBV,165.49,288235706659,N/A


In [10]:
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,140.98,40859937750,N/A
1,AAL,19.02,12350649702,N/A
2,AAP,225.79,13952515186,N/A
3,AAPL,176.9,2875536657357,N/A
4,ABBV,146.903,265740650248,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [30]:
# split tickers into sub-lists of length 100
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

ticker_grps = list(chunks(stocks['Ticker'], 100))
len(ticker_grps)


6

In [31]:
ticker_str = []
for i in range(0, len(ticker_grps)):
    ticker_str.append(','.join(ticker_grps[i]))

ticker_str

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

In [38]:
from pprint import pprint

for tickers in ticker_str:
    batchendpoint = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={tickers}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batchendpoint).json()
    pprint(data)
    
    for ticker in tickers.split(','):
        df = df.append(pd.Series(
        [
        stock,
        data[ticker]['quote']['latestPrice'],
        data[ticker]['quote']['marketCap'],
        "N/A"
        ], index=my_columns),ignore_index=True)

{'A': {'quote': {'avgTotalVolume': 1661588,
                 'calculationPrice': 'close',
                 'change': -0.77,
                 'changePercent': -0.0057,
                 'close': 0,
                 'closeSource': 'lcfifiao',
                 'closeTime': None,
                 'companyName': 'Agilent Technologies Inc.',
                 'currency': 'USD',
                 'delayedPrice': None,
                 'delayedPriceTime': None,
                 'extendedChange': None,
                 'extendedChangePercent': None,
                 'extendedPrice': None,
                 'extendedPriceTime': None,
                 'high': 0,
                 'highSource': 'ietem5epeud ray1nild  c',
                 'highTime': 1707161578321,
                 'iexAskPrice': 0,
                 'iexAskSize': 0,
                 'iexBidPrice': 0,
                 'iexBidSize': 0,
                 'iexClose': 136.4,
                 'iexCloseTime': 1687889181467,
                 'ie

/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipyke

{'CME': {'quote': {'avgTotalVolume': 2173318,
                   'calculationPrice': 'iexlasttrade',
                   'change': 0.03,
                   'changePercent': 0.00013,
                   'close': 0,
                   'closeSource': 'folifaci',
                   'closeTime': None,
                   'companyName': 'CME Group Inc - Class A',
                   'currency': 'USD',
                   'delayedPrice': None,
                   'delayedPriceTime': None,
                   'extendedChange': None,
                   'extendedChangePercent': None,
                   'extendedPrice': None,
                   'extendedPriceTime': None,
                   'high': 0,
                   'highSource': None,
                   'highTime': None,
                   'iexAskPrice': 0,
                   'iexAskSize': 0,
                   'iexBidPrice': 0,
                   'iexBidSize': 0,
                   'iexClose': 247.77,
                   'iexCloseTime': 165516348619

/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipyke

{'GIS': {'quote': {'avgTotalVolume': 3723668,
                   'calculationPrice': 'close',
                   'change': -0.25,
                   'changePercent': -0.00363,
                   'close': 0,
                   'closeSource': 'iaoclffi',
                   'closeTime': None,
                   'companyName': 'General Mills, Inc.',
                   'currency': 'USD',
                   'delayedPrice': None,
                   'delayedPriceTime': None,
                   'extendedChange': None,
                   'extendedChangePercent': None,
                   'extendedPrice': None,
                   'extendedPriceTime': None,
                   'high': 0,
                   'highSource': 'delmtadniy c erup1ie e5',
                   'highTime': 1663675835196,
                   'iexAskPrice': 0,
                   'iexAskSize': 0,
                   'iexBidPrice': 0,
                   'iexBidSize': 0,
                   'iexClose': 68.92,
                   'iexClos

...

{'MCK': {'quote': {'avgTotalVolume': 1215250,
                   'calculationPrice': 'close',
                   'change': 1.97,
                   'changePercent': 0.00718,
                   'close': 0,
                   'closeSource': 'ilffaoic',
                   'closeTime': None,
                   'companyName': 'Mckesson Corporation',
                   'currency': 'USD',
                   'delayedPrice': None,
                   'delayedPriceTime': None,
                   'extendedChange': None,
                   'extendedChangePercent': None,
                   'extendedPrice': None,
                   'extendedPriceTime': None,
                   'high': 0,
                   'highSource': 'eedde arlpt1mu ceii y5n',
                   'highTime': 1653417950057,
                   'iexAskPrice': 0,
                   'iexAskSize': 0,
                   'iexBidPrice': 0,
                   'iexBidSize': 0,
                   'iexClose': 278.04,
                   'iexClos

/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipyke

{'ROL': {'quote': {'avgTotalVolume': 2668294,
                   'calculationPrice': 'close',
                   'change': 0.07,
                   'changePercent': 0.00225,
                   'close': 0,
                   'closeSource': 'iiaofclf',
                   'closeTime': None,
                   'companyName': 'Rollins, Inc.',
                   'currency': 'USD',
                   'delayedPrice': None,
                   'delayedPriceTime': None,
                   'extendedChange': None,
                   'extendedChangePercent': None,
                   'extendedPrice': None,
                   'extendedPriceTime': None,
                   'high': 0,
                   'highSource': 'mye iecld1d5pi  eatrune',
                   'highTime': 1660965980788,
                   'iexAskPrice': 0,
                   'iexAskSize': 0,
                   'iexBidPrice': 0,
                   'iexBidSize': 0,
                   'iexClose': 31.76,
                   'iexCloseTime': 

/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipykernel_6331/2318564533.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(
/tmp/ipyke

KeyError: 'WLTW'

In [39]:
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,140.98,40859937750,N/A
1,AAL,19.02,12350649702,N/A
2,AAP,225.79,13952515186,N/A
3,AAPL,176.9,2875536657357,N/A
4,ABBV,146.903,265740650248,N/A
...,...,...,...,...
484,ABBV,57.3,18301757511,N/A
485,ABBV,92.98,28812445228,N/A
486,ABBV,87.41,36827748119,N/A
487,ABBV,59.98,248669127402,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: